In [18]:
import os
import pandas as pd
import numpy as np
from scipy.stats import kstest

In [19]:
dicts = []


folder = 'final_data'
for file_name in os.listdir(folder):
    file_path = '/'.join([folder, file_name])
    with open(file_path, 'r') as file:
        full_file = file.read()
        full_file = full_file.replace('Full results', 'Full_results')
        split = full_file.split('\n\n')
        for sub_file in split[:2]:
            the_dict = {}
            for line in sub_file.splitlines():
                name, value = line.split(' ', 1)
                the_dict[name] = value
            dicts.append(the_dict)

In [20]:
l = []

for i in range(0, len(dicts), 2):
    paths2vec = dicts[i]
    random2vec = dicts[i + 1]

    l.append((paths2vec['dataset:'], paths2vec['Full_results:'], random2vec['Full_results:'], paths2vec['metric:'], paths2vec['runs:'], paths2vec['subset:']))


In [21]:
new_l = []
for i in range(len(l)):
    paths = [float(i) for i in l[i][1].replace('[', '').replace(']', '').replace(',', '').split()]
    rands = [float(i) for i in l[i][2].replace('[', '').replace(']', '').replace(',', '').split()]
    new_l.append((l[i][0], paths, rands, l[i][3], l[i][4], l[i][5]))

In [22]:
df = []
for name, paths, rand, metric, runs, subset_name in new_l:
    df.append([
        name,
        subset_name,
        runs,
        metric,
        np.mean(paths),
        np.std(paths),
        np.mean(rand),
        np.std(rand),
        kstest(paths, rand).statistic,
        kstest(paths, rand).pvalue
        ])
df = pd.DataFrame(df, columns=['name', 'subset_name', 'runs', 'metric', 'paths2vec mean', 'paths2vec std', 'random mean', 'random std', 'kstest statistic', 'kstest pvalue'])
df

,name,subset_name,runs,metric,paths2vec mean,paths2vec std,random mean,random std,kstest statistic,kstest pvalue
0,ogbg-molbace,valid,10,rocauc,0.644267,0.037949,0.492436,0.025365,1.000000,0.000011
1,ogbg-molbbbp,valid,10,rocauc,0.746092,0.012989,0.500000,0.000000,1.000000,0.000011
2,ogbg-molclintox,valid,10,rocauc,0.500000,0.000000,0.500000,0.000000,0.000000,1.000000
3,ogbg-molesol,valid,10,rmse,1.488492,0.021047,2.294547,0.029359,1.000000,0.000011
4,ogbg-molfreesolv,valid,10,rmse,5.792643,0.082057,6.767195,0.038776,1.000000,0.000011
5,ogbg-molhiv,valid,3,rocauc,0.505966,0.005040,0.500000,0.000000,0.666667,0.600000
6,ogbg-mollipo,valid,10,rmse,1.132198,0.010670,1.247445,0.010036,1.000000,0.000011
7,ogbg-molmuv,valid,3,ap,0.002260,0.000000,0.002260,0.000000,0.000000,1.000000
8,ogbg-molsider,valid,10,rocauc,0.509036,0.001612,0.500366,0.002665,0.900000,0.000217
9,ogbg-moltox21,valid,10,rocauc,0.500038,0.000113,0.500000,0.000000,0.100000,1.000000


In [23]:
df['paths2vec mean'] = df['paths2vec mean'].round(2).astype(str).map(lambda x: f'{x:<04}')
df['paths2vec std'] = df['paths2vec std'].round(4).astype(str).map(lambda x: f'{x:<06}')
df['random mean'] = df['random mean'].round(2).astype(str).map(lambda x: f'{x:<04}')
df['random std'] = df['random std'].round(4).astype(str).map(lambda x: f'{x:<06}')
df['kstest statistic'] = df['kstest statistic'].round(2)
df['kstest pvalue'] = df['kstest pvalue'].round(5)
df

,name,subset_name,runs,metric,paths2vec mean,paths2vec std,random mean,random std,kstest statistic,kstest pvalue
0,ogbg-molbace,valid,10,rocauc,0.64,0.0379,0.49,0.0254,1.00,0.00001
1,ogbg-molbbbp,valid,10,rocauc,0.75,0.0130,0.50,0.0000,1.00,0.00001
2,ogbg-molclintox,valid,10,rocauc,0.50,0.0000,0.50,0.0000,0.00,1.00000
3,ogbg-molesol,valid,10,rmse,1.49,0.0210,2.29,0.0294,1.00,0.00001
4,ogbg-molfreesolv,valid,10,rmse,5.79,0.0821,6.77,0.0388,1.00,0.00001
5,ogbg-molhiv,valid,3,rocauc,0.51,0.0050,0.50,0.0000,0.67,0.60000
6,ogbg-mollipo,valid,10,rmse,1.13,0.0107,1.25,0.0100,1.00,0.00001
7,ogbg-molmuv,valid,3,ap,0.00,0.0000,0.00,0.0000,0.00,1.00000
8,ogbg-molsider,valid,10,rocauc,0.51,0.0016,0.50,0.0027,0.90,0.00022
9,ogbg-moltox21,valid,10,rocauc,0.50,0.0001,0.50,0.0000,0.10,1.00000


In [24]:
table_df = pd.DataFrame()
table_df['name'] = df ['name']
table_df['runs'] = df ['runs']
table_df['metric'] = df ['metric']
table_df['paths2vec'] = df ['paths2vec mean'] + " +- " + df['paths2vec std']
table_df['random'] = df ['random mean'] + " +- " + df['random std']

In [25]:
regression_df = table_df[table_df['metric'] == 'rmse']
classification_df = table_df[table_df['metric'] != 'rmse'].drop(labels = ['random'], axis=1)

In [26]:
classification_df

,name,runs,metric,paths2vec
0,ogbg-molbace,10,rocauc,0.64 +- 0.0379
1,ogbg-molbbbp,10,rocauc,0.75 +- 0.0130
2,ogbg-molclintox,10,rocauc,0.50 +- 0.0000
5,ogbg-molhiv,3,rocauc,0.51 +- 0.0050
7,ogbg-molmuv,3,ap,0.00 +- 0.0000
8,ogbg-molsider,10,rocauc,0.51 +- 0.0016
9,ogbg-moltox21,10,rocauc,0.50 +- 0.0001
10,ogbg-moltoxcast,10,rocauc,0.50 +- 0.0001


In [27]:
regression_df

,name,runs,metric,paths2vec,random
3,ogbg-molesol,10,rmse,1.49 +- 0.0210,2.29 +- 0.0294
4,ogbg-molfreesolv,10,rmse,5.79 +- 0.0821,6.77 +- 0.0388
6,ogbg-mollipo,10,rmse,1.13 +- 0.0107,1.25 +- 0.0100


In [28]:
print(classification_df.to_latex(index=False))

\begin{tabular}{llll}
\toprule
           name & runs & metric &      paths2vec \\
\midrule
   ogbg-molbace &   10 & rocauc & 0.64 +- 0.0379 \\
   ogbg-molbbbp &   10 & rocauc & 0.75 +- 0.0130 \\
ogbg-molclintox &   10 & rocauc & 0.50 +- 0.0000 \\
    ogbg-molhiv &    3 & rocauc & 0.51 +- 0.0050 \\
    ogbg-molmuv &    3 &     ap & 0.00 +- 0.0000 \\
  ogbg-molsider &   10 & rocauc & 0.51 +- 0.0016 \\
  ogbg-moltox21 &   10 & rocauc & 0.50 +- 0.0001 \\
ogbg-moltoxcast &   10 & rocauc & 0.50 +- 0.0001 \\
\bottomrule
\end{tabular}



In [29]:
print(regression_df.to_latex(index=False))

\begin{tabular}{lllll}
\toprule
            name & runs & metric &      paths2vec &         random \\
\midrule
    ogbg-molesol &   10 &   rmse & 1.49 +- 0.0210 & 2.29 +- 0.0294 \\
ogbg-molfreesolv &   10 &   rmse & 5.79 +- 0.0821 & 6.77 +- 0.0388 \\
    ogbg-mollipo &   10 &   rmse & 1.13 +- 0.0107 & 1.25 +- 0.0100 \\
\bottomrule
\end{tabular}

